<a href="https://colab.research.google.com/github/ArafathAli7/CV-PROJECT/blob/main/fakenewsdetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-learn


In [3]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset


Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
 83% 34.0M/41.0M [00:00<00:00, 184MB/s]
100% 41.0M/41.0M [00:00<00:00, 181MB/s]


In [8]:
import zipfile
with zipfile.ZipFile('fake-and-real-news-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('news_dataset')


In [10]:
import os
print(os.listdir('news_dataset'))



['Fake.csv', 'True.csv']


In [11]:
import pandas as pd

df_fake = pd.read_csv('news_dataset/Fake.csv')
df_real = pd.read_csv('news_dataset/True.csv')

df_fake['label'] = 0  # Fake news
df_real['label'] = 1  # Real news

df = pd.concat([df_fake, df_real]).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle dataset

print(df.head())



                                               title  \
0  IRS identifies more cyberattacks to access tax...   
1  Trump budget cuts may stir backlash in rural A...   
2   Watch Trump Mock His Moronic Supporters To Th...   
3  WHO NEEDS NANCY PELOSI WHEN CONGRESS HAS PAUL ...   
4  POPE MEETS WITH ANGRY, JEWISH, PR0 LATE-TERM A...   

                                                text       subject  \
0  (Reuters) - The Internal Revenue Service said ...  politicsNews   
1  WASHINGTON (Reuters) - President Donald Trump’...  politicsNews   
2  If there s one thing Donald Trump knows how to...          News   
3  Why do we even need a Congress anymore? We hav...     left-news   
4  As a Catholic I m offended by this Pope s poli...     left-news   

                 date  label  
0  February 26, 2016       1  
1       May 23, 2017       1  
2     January 6, 2016      0  
3        Jun 11, 2015      0  
4        Apr 16, 2016      0  


In [14]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

# Convert text to numerical features using TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train the model using Logistic Regression
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict and check accuracy
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 98.80%


In [15]:
def predict_news(news):
    news_cleaned = clean_text(news)  # Clean input text
    news_vectorized = vectorizer.transform([news_cleaned])  # Convert to TF-IDF
    prediction = model.predict(news_vectorized)  # Predict
    return "Real News" if prediction[0] == 1 else "Fake News"

# Test Example
test_news = "Breaking: The government has launched a new policy for healthcare."
print(f"Prediction: {predict_news(test_news)}")


Prediction: Real News
